In [ ]:
import molsysmt as msm

In [ ]:
prmtop_file = msm.demo['pentalanine']['pentalanine.prmtop']
inpcrd_file = msm.demo['pentalanine']['pentalanine.inpcrd']

In [ ]:
molsys = msm.convert([prmtop_file, inpcrd_file], to_form='molsysmt.MolSys')

In [ ]:
aa = msm.convert([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], to_form='molsysmt.MolecularMechanics')

In [ ]:
aa.forcefield

In [ ]:
msm.has_attribute([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], 'kappa')

In [ ]:
msm.where_is_attribute([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], 'kappa')

In [ ]:
molsys = msm.build.solvate([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                   box_shape='truncated octahedral', clearance='14.0 angstroms',
                   to_form='molsysmt.MolSys', engine="OpenMM")

In [ ]:
molsys.molecular_mechanics.flexible_constraints